# Demo of RtDeep

We define a simple network using RtDeep with microcircuits.

Task: learn to reproduce output of a teacher network.

The model uses real time dynamics with Euler integration, LE, dendritic error propagation and a phased implementation of DTP-DRL.

## Definitions

In [1]:
%matplotlib widget

import numpy as np
import pylab as plt
plt.rc('text', usetex=True)
plt.rc('font', size=12,family='serif')
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
from microcircuit import *
import sys
import pandas as pd


#load autoreload, which automatically reloads the microcircuit.py upon execution
%reload_ext autoreload
%autoreload 1
%aimport microcircuit

## Adding noise, and filtering it out again

We add phased noise to one layer and filter it out again, s.t. synaptic plasticity remains the same.

### Setup

In [33]:
# simulation settings

dt     = 1e-4 # in ms
dtxi   = 10 * dt
tausyn = 100 * dt
Tpres  = 1e+4 * dt
Tbw    = 1 * Tpres # time scale of phases for backward weight learning
Ttot   = 30 * Tpres # total simulation time

# noise settings
noise_scale = [5*1e-7]

In [34]:
np.random.seed(1234)

# Init model: weights, voltages

layers = [1, 1, 1]

# conductances
gl = 0.03
gbas = 0.1
gapi = 0.06
gden = 0.1
gnI = 0.06
gntgt = 0.06

# learning rates in 1/ms
eta_fw = [0.0] * (len(layers) - 1)
eta_bw = [200.0] * (len(layers) - 2)
eta_IP = [40.0] * (len(layers) - 1)
eta_PI = [50.0] * (len(layers) - 1)

# regularizer for backward weight learning
alpha = [1e-3]

# initialise voltages
uP_init = []
for i in range(1, len(layers)):
    uP_init.append(np.random.normal(0, 1, size=layers[i]))
    
uI_init = [np.random.normal(0, 1, size=layers[-1])]
    
# forward pp weights: connects all layers k, k+1
WPP_init = []
for i in range(len(layers)-1):
    WPP_init.append(np.random.uniform(-1, 1, size=(layers[i+1], layers[i])))

# p to inter: connects N-1 to N
WIP_init = [np.random.uniform(-1, 1, size=(layers[-1], layers[-2]))]

# backwards p to p: connects N to k
BPP_init = []
for i in range(1, len(layers)-1):
    BPP_init.append(np.random.uniform(-1, 1, size=(layers[i], layers[-1])))

# backwards inter to p: connects N to k
BPI_init = []
for i in range(1, len(layers)-1):
    BPI_init.append(np.random.uniform(-1, 1, size=(layers[i], layers[-1])))

In [35]:
# input rates: step functions in the form of random inputs held for Tpres

r0_random = np.random.uniform(0, 1, size=(int(Ttot / Tpres), layers[0]))
r0_random = np.repeat(r0_random, int(Tpres / dt), axis=0)

In [36]:
mc1 = phased_noise_model(dt, dtxi, tausyn, Tbw, Tpres, noise_scale, alpha, 'LDRL', logistic, d_logistic, layers,
            uP_init=uP_init, uI_init=uI_init,
            WPP_init=WPP_init, WIP_init=WIP_init, BPP_init=BPP_init, BPI_init=BPI_init,
            gl=gl, gden=gden, gbas=gbas, gapi=gapi, gnI=gnI, gntgt=gntgt,
            eta_fw=eta_fw, eta_bw=eta_bw, eta_PI=eta_PI, eta_IP=eta_IP)

# mc1.set_self_predicting_state()

### Simulation

In [37]:
# # pre-training to settle voltages
# for i in range(len(r0_random)):
#     mc1.evolve_system(r0=r0_random[i], learn_weights=False, learn_bw_weights=False)

In [ ]:
u_time_series = []
uIbreve_time_series = []
uPbreve_time_series = []
rIbreve_time_series = []
rPbreve_time_series = []
rPbreve_HI_time_series = []

WPP_time_series = []
BPP_time_series = []
WIP_time_series = []
BPI_time_series = []

for i in range(len(r0_random)):
    mc1.evolve_system(r0=r0_random[i], learn_weights=True, learn_bw_weights=True)
    
    uIbreve_time_series.append(copy.deepcopy(mc1.uI_breve))
    uPbreve_time_series.append(copy.deepcopy(mc1.uP_breve))
    rIbreve_time_series.append(copy.deepcopy(mc1.rI_breve))
    rPbreve_time_series.append(copy.deepcopy(mc1.rP_breve))
    u_time_series.append(copy.deepcopy(mc1.get_voltages()))

    rPbreve_HI_time_series.append(copy.deepcopy(mc1.rP_breve_HI))
    WPP, WIP, BPP, BPI = mc1.get_weights()
    WPP_time_series.append(copy.deepcopy(WPP))
    BPP_time_series.append(copy.deepcopy(BPP))
    WIP_time_series.append(copy.deepcopy(WIP))
    BPI_time_series.append(copy.deepcopy(BPI))
        
uP_time_series = [u[0] for u in u_time_series]
uI_time_series = [u[1] for u in u_time_series]

Current time: 0s
Learning backward weights to layer 1
Current time: 1.0s
Learning backward weights to layer 1
Current time: 2.0s
Learning backward weights to layer 1


In [ ]:
fig = plt.figure()
uP = [uP[0] for uP in uPbreve_time_series]
plt.plot(np.array(uP[int(Tpres/dt):]).ravel())
plt.show()

fig = plt.figure()
uP = [uP[1] for uP in uPbreve_time_series]
plt.plot(np.array(uP[int(Tpres/dt):]).ravel())
plt.plot(np.array(uIbreve_time_series[int(Tpres/dt):]).ravel())
plt.xlabel('dt')
plt.show()


fig = plt.figure()
rPbreve_HI = [rPbreve_HI[1] for rPbreve_HI in rPbreve_HI_time_series]
plt.plot(np.array(rPbreve_HI[int(Tpres/dt):]).ravel(), label="rPbreveHI")
plt.xlabel('dt')
plt.legend()
plt.show()

Now, let's also check that BPI has converged onto -BPP and WIP onto WPP:

In [ ]:
len(BPP_time_series)

In [ ]:
cos_BPI_BPP = []
for i in range(len(BPI_time_series[0])):
    cos_BPI_BPP.append(np.array([
        cos_sim(BPI_time_series[j][i], -BPP_time_series[j][i]) for j in range(len(BPI_time_series))
                           ]))
    
cos_WIP_WPP = []
for i in range(len(WIP_time_series[0])):
    cos_WIP_WPP.append(np.array([
        cos_sim(WIP_time_series[j][i], WPP_time_series[j][i+1]) for j in range(len(WIP_time_series))
                           ]))
    
cos_WPPtrans_BPP = []
for i in range(len(BPP_time_series[0])):
    cos_WPPtrans_BPP.append(np.array([
        cos_sim(BPP_time_series[j][i], WPP_time_series[j][i+1].T) for j in range(len(BPP_time_series))
                           ]))

In [ ]:
for i in range(len(layers[1:-1])):
    fig, ax = plt.subplots()
    ax.plot(cos_WPPtrans_BPP[i])
    plt.yscale('linear')
    label = 'dt'
    plt.xlabel(label)
    plt.title('$\cos(B^\mathrm{PP}, (W^\mathrm{PP})^T)$ to layer ' + str(i+1))
    plt.show()
    

fig, ax = plt.subplots()
ax.plot(np.array([WPP[1].T for WPP in WPP_time_series]).ravel(), label="WPP.T")
ax.plot(np.array(BPP_time_series).ravel(), label="BPP")
plt.yscale('linear')
label = 'dt'
plt.xlabel(label)
plt.ylabel("B")
plt.legend()
# plt.title('$\cos(W^\mathrm{IP}, W^\mathrm{PP})$ to layer ' + str(i+1))
plt.show()

In [ ]:
for i in range(len(layers[1:-1])):
    fig, ax = plt.subplots()
    ax.plot(cos_BPI_BPP[i])
    plt.yscale('linear')
    label = 'dt'
    plt.xlabel(label)
    plt.title('$\cos(B^\mathrm{PI}, -B^\mathrm{PP})$ to layer ' + str(i+1))
    plt.show()
    

fig, ax = plt.subplots()
ax.plot(np.array(BPI_time_series).ravel(), label="BPI")
ax.plot(-np.array(BPP_time_series).ravel(), label="BPP")
plt.yscale('linear')
label = 'dt'
plt.xlabel(label)
plt.ylabel("B")
plt.legend()
# plt.title('$\cos(W^\mathrm{IP}, W^\mathrm{PP})$ to layer ' + str(i+1))
plt.show()

In [ ]:
for i in range(len(layers[1:-1])):
    fig, ax = plt.subplots()
    ax.plot(cos_WIP_WPP[i])
    plt.yscale('linear')
    label = 'dt'
    plt.xlabel(label)
    plt.title('$\cos(W^\mathrm{IP}, W^\mathrm{PP})$ to layer ' + str(i+1))
    plt.show()


fig, ax = plt.subplots()
ax.plot(np.array([WPP[1] for WPP in WPP_time_series]).ravel())
ax.plot(np.array(WIP_time_series).ravel())
plt.yscale('linear')
label = 'dt'
# plt.legend()
plt.xlabel(label)
plt.show()